In [2]:
import boto3
import pandas as pd
from io import StringIO

In [3]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')



In [4]:
bucket_obj_1 = bucket.objects.filter(Prefix='2022-03-15/')
bucket_obj_2 = bucket.objects.filter(Prefix='2022-02-10/')

In [5]:
objects= [ obj for obj in bucket_obj_1 ] + [ obj for obj in bucket_obj_2 ]

In [6]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR10.csv'),
 s3.Object

In [7]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data,delimiter=',')

In [8]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [9]:
df_all = pd.DataFrame(columns=df_init.columns)
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades


In [31]:
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter=',')
    if not df.empty and not df.isna().all().all():
        df_all = pd.concat([df_all, df], ignore_index=True)

In [11]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,35
1,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537,37
2,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585,29
3,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539,8
4,DE000A0LD6E6,GXI,GERRESHEIMER AG,Common stock,EUR,2504380,2022-03-15,08:00,61.40,61.40,60.95,61.05,329,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238089,DE0007664005,VOW,VOLKSWAGEN AG ST O.N.,Common stock,EUR,2505113,2022-02-10,16:44,254.80,254.80,254.80,254.80,40,2
238090,LU1899270539,ECBI,SI UC.-UC M.E.G.B.E. EOA,ETF,EUR,3629672,2022-02-10,16:53,101.20,101.20,101.20,101.20,45,2
238091,US03073E1055,ABG,"AMERISOURCEBERGEN DL-,01",Common stock,EUR,5988600,2022-02-10,20:30,124.00,124.00,124.00,124.00,0,1
238092,US25746U1097,DOD,DOMINION ENERGY INC.,Common stock,EUR,5990125,2022-02-10,20:30,70.14,70.14,70.14,70.14,0,1


In [12]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [13]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329
...,...,...,...,...,...,...,...,...
238089,DE0007664005,2022-02-10,16:44,254.80,254.80,254.80,254.80,40
238090,LU1899270539,2022-02-10,16:53,101.20,101.20,101.20,101.20,45
238091,US03073E1055,2022-02-10,20:30,124.00,124.00,124.00,124.00,0
238092,US25746U1097,2022-02-10,20:30,70.14,70.14,70.14,70.14,0


In [14]:
df_all.dropna(inplace=True)
#Drops any row if it contains null in any of the columns

In [15]:
df_all.shape

(238094, 8)

#Get opening price per ISIN and day

In [16]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [17]:
df_all[df_all['ISIN']=='DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34
158,DE000A0DJ6J9,2022-03-15,08:01,37.28,37.28,37.20,37.20,2157,37.34
294,DE000A0DJ6J9,2022-03-15,08:02,37.14,37.28,37.06,37.06,2284,37.34
578,DE000A0DJ6J9,2022-03-15,08:03,37.06,37.06,36.92,36.94,1608,37.34
2999,DE000A0DJ6J9,2022-03-15,08:05,36.90,36.90,36.78,36.78,2449,37.34
...,...,...,...,...,...,...,...,...,...
233178,DE000A0DJ6J9,2022-02-10,16:25,30.92,30.92,30.90,30.90,229,31.44
233465,DE000A0DJ6J9,2022-02-10,16:26,30.92,30.92,30.92,30.92,33,31.44
233739,DE000A0DJ6J9,2022-02-10,16:27,30.90,30.90,30.90,30.90,173,31.44
234307,DE000A0DJ6J9,2022-02-10,16:29,30.94,30.94,30.94,30.94,5,31.44


#Get opening price per ISIN and day

In [18]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [19]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34,37.06
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537,16.58,16.40
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585,201.10,204.70
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539,62.55,61.30
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329,61.40,61.45
...,...,...,...,...,...,...,...,...,...,...
238089,DE0007664005,2022-02-10,16:44,254.80,254.80,254.80,254.80,40,258.60,254.80
238090,LU1899270539,2022-02-10,16:53,101.20,101.20,101.20,101.20,45,102.99,101.20
238091,US03073E1055,2022-02-10,20:30,124.00,124.00,124.00,124.00,0,124.00,124.00
238092,US25746U1097,2022-02-10,20:30,70.14,70.14,70.14,70.14,0,70.14,70.14


In [20]:
df_all[df_all['ISIN']=='DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34,37.06
158,DE000A0DJ6J9,2022-03-15,08:01,37.28,37.28,37.20,37.20,2157,37.34,37.06
294,DE000A0DJ6J9,2022-03-15,08:02,37.14,37.28,37.06,37.06,2284,37.34,37.06
578,DE000A0DJ6J9,2022-03-15,08:03,37.06,37.06,36.92,36.94,1608,37.34,37.06
2999,DE000A0DJ6J9,2022-03-15,08:05,36.90,36.90,36.78,36.78,2449,37.34,37.06
...,...,...,...,...,...,...,...,...,...,...
233178,DE000A0DJ6J9,2022-02-10,16:25,30.92,30.92,30.90,30.90,229,31.44,30.92
233465,DE000A0DJ6J9,2022-02-10,16:26,30.92,30.92,30.92,30.92,33,31.44,30.92
233739,DE000A0DJ6J9,2022-02-10,16:27,30.90,30.90,30.90,30.90,173,31.44,30.92
234307,DE000A0DJ6J9,2022-02-10,16:29,30.94,30.94,30.94,30.94,5,31.44,30.92


#Aggregations

In [21]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [22]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-02-10,38.950,38.9500,38.950,39.4500,147
1,AT000000STR1,2022-03-15,36.400,37.0500,36.400,37.5500,536
2,AT00000FACC2,2022-02-10,9.300,9.2500,9.220,9.4300,620
3,AT00000FACC2,2022-03-15,7.880,7.7100,7.610,7.8800,68
4,AT0000606306,2022-02-10,28.220,28.2000,27.960,28.3600,4765
...,...,...,...,...,...,...,...
6457,XS2427363036,2022-03-15,21.490,21.4900,21.490,21.4900,0
6458,XS2427363895,2022-03-15,18.200,18.1900,18.190,18.2000,0
6459,XS2427474023,2022-03-15,25.096,25.0960,25.096,25.0960,0
6460,XS2434891219,2022-03-15,3.484,3.5828,3.484,3.5828,700


In [23]:
#Percent Change Previous Closing

In [24]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [25]:
df_all.sort_values(by=['ISIN','Date'])

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-02-10,38.950,38.9500,38.950,39.4500,147,NaN
1,AT000000STR1,2022-03-15,36.400,37.0500,36.400,37.5500,536,38.95
2,AT00000FACC2,2022-02-10,9.300,9.2500,9.220,9.4300,620,NaN
3,AT00000FACC2,2022-03-15,7.880,7.7100,7.610,7.8800,68,9.25
4,AT0000606306,2022-02-10,28.220,28.2000,27.960,28.3600,4765,NaN
...,...,...,...,...,...,...,...,...
6457,XS2427363036,2022-03-15,21.490,21.4900,21.490,21.4900,0,NaN
6458,XS2427363895,2022-03-15,18.200,18.1900,18.190,18.2000,0,NaN
6459,XS2427474023,2022-03-15,25.096,25.0960,25.096,25.0960,0,NaN
6460,XS2434891219,2022-03-15,3.484,3.5828,3.484,3.5828,700,NaN


In [26]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [27]:
df_all.sort_values(by=['ISIN','Date'])

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price,change_prev_closing_%
0,AT000000STR1,2022-02-10,38.950,38.9500,38.950,39.4500,147,NaN,NaN
1,AT000000STR1,2022-03-15,36.400,37.0500,36.400,37.5500,536,38.95,-4.878049
2,AT00000FACC2,2022-02-10,9.300,9.2500,9.220,9.4300,620,NaN,NaN
3,AT00000FACC2,2022-03-15,7.880,7.7100,7.610,7.8800,68,9.25,-16.648649
4,AT0000606306,2022-02-10,28.220,28.2000,27.960,28.3600,4765,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6457,XS2427363036,2022-03-15,21.490,21.4900,21.490,21.4900,0,NaN,NaN
6458,XS2427363895,2022-03-15,18.200,18.1900,18.190,18.2000,0,NaN,NaN
6459,XS2427474023,2022-03-15,25.096,25.0960,25.096,25.0960,0,NaN,NaN
6460,XS2434891219,2022-03-15,3.484,3.5828,3.484,3.5828,700,NaN,NaN


In [28]:
df_all.drop(columns=['prev_closing_price'], inplace=True)
df_all = df_all.round(decimals=2)

In [30]:
df_all.sort_values(by=['ISIN','Date'])

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-02-10,38.95,38.95,38.95,39.45,147,NaN
1,AT000000STR1,2022-03-15,36.40,37.05,36.40,37.55,536,-4.88
2,AT00000FACC2,2022-02-10,9.30,9.25,9.22,9.43,620,NaN
3,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.88,68,-16.65
4,AT0000606306,2022-02-10,28.22,28.20,27.96,28.36,4765,NaN
...,...,...,...,...,...,...,...,...
6457,XS2427363036,2022-03-15,21.49,21.49,21.49,21.49,0,NaN
6458,XS2427363895,2022-03-15,18.20,18.19,18.19,18.20,0,NaN
6459,XS2427474023,2022-03-15,25.10,25.10,25.10,25.10,0,NaN
6460,XS2434891219,2022-03-15,3.48,3.58,3.48,3.58,700,NaN
